In [1]:
import cv2
import numpy as np
import pyautogui
from ultralytics import YOLO

In [2]:
# Load your YOLO model
model_path = 'Yolos/Honeycomb/train17/weights/best.pt'  # Path to your custom YOLO model
model = YOLO(model_path)

In [3]:
# Set the detection threshold
threshold = 0.2

In [4]:
# Get screen dimensions
screen_width, screen_height = pyautogui.size()

# Define region for screen capture (full screen)
region = (0, 0, screen_width, screen_height)

# Create an OpenCV window and set its properties
window_name = "YOLO Detection on Screen"
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)  # Allow manual resizing
cv2.resizeWindow(window_name, 800, 600)  # Set the initial size of the window

while True:
    # Capture the screen
    screenshot = pyautogui.screenshot(region=region)
    frame = np.array(screenshot)

    # Convert colors from PIL to OpenCV (RGB to BGR)
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    # Run YOLO model on the frame
    results = model.predict(source=frame, conf=threshold, verbose=False)

    detections = results[0].boxes.data.cpu().numpy()  # Assuming this returns [x1, y1, x2, y2, confidence, class_id]
    class_counts = {}
    for detection in detections:
        class_id = int(detection[5])  # Get the class ID
        class_counts[class_id] = class_counts.get(class_id, 0) + 1
        
    # Annotate detections on the frame
    annotated_frame = results[0].plot()
    
    y_offset = 20
    for class_id, count in class_counts.items():
        class_name = model.names[class_id]  # Assuming `model.names` maps class IDs to names
        text = f"{class_name}: {count}"
        cv2.putText(
            annotated_frame,
            text,
            (10, y_offset),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,
            (0, 255, 0),
            2,
            cv2.LINE_AA,
        )
        y_offset += 30
    # Display the frame in the resized window
    cv2.imshow(window_name, annotated_frame)

    # Exit when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cleanup
cv2.destroyAllWindows()